In [1]:
import numpy as np
import os
import json
import cv2
import random
import sys
import matplotlib.pyplot as plt

import detectron2
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

sys.path.append("../src")
from defs import RAW_DATA_PATH

In [2]:
from detectron2.data.datasets import load_coco_json, load_sem_seg
from detectron2.data.datasets.coco_panoptic import merge_to_panoptic

def register_coco_panoptic_separated(
    name, metadata, image_root, panoptic_root, panoptic_json, sem_seg_root, instances_json
):
    """
    Register a "separated" version of COCO panoptic segmentation dataset named `name`.
    The annotations in this registered dataset will contain both instance annotations and
    semantic annotations, each with its own contiguous ids. Hence it's called "separated".
    It follows the setting used by the PanopticFPN paper:
    1. The instance annotations directly come from polygons in the COCO
       instances annotation task, rather than from the masks in the COCO panoptic annotations.
       The two format have small differences:
       Polygons in the instance annotations may have overlaps.
       The mask annotations are produced by labeling the overlapped polygons
       with depth ordering.
    2. The semantic annotations are converted from panoptic annotations, where
       all "things" are assigned a semantic id of 0.
       All semantic categories will therefore have ids in contiguous
       range [1, #stuff_categories].
    This function will also register a pure semantic segmentation dataset
    named ``name + '_stuffonly'``.
    Args:
        name (str): the name that identifies a dataset,
            e.g. "coco_2017_train_panoptic"
        metadata (dict): extra metadata associated with this dataset.
        image_root (str): directory which contains all the images
        panoptic_root (str): directory which contains panoptic annotation images
        panoptic_json (str): path to the json panoptic annotation file
        sem_seg_root (str): directory which contains all the ground truth segmentation annotations.
        instances_json (str): path to the json instance annotation file
    """
    panoptic_name = name + "_separated"
    DatasetCatalog.register(
        panoptic_name,
        lambda: merge_to_panoptic(
            load_coco_json(instances_json, image_root, panoptic_name),
            load_sem_seg(sem_seg_root, image_root, image_ext="png"),
        ),
    )
    MetadataCatalog.get(panoptic_name).set(
        panoptic_root=panoptic_root,
        image_root=image_root,
        panoptic_json=panoptic_json,
        sem_seg_root=sem_seg_root,
        json_file=instances_json,  # TODO rename
        evaluator_type="coco_panoptic_seg",
        ignore_label=255,
        **metadata,
    )

    semantic_name = name + "_stuffonly"
    DatasetCatalog.register(semantic_name, lambda: load_sem_seg(sem_seg_root, image_root))
    MetadataCatalog.get(semantic_name).set(
        sem_seg_root=sem_seg_root,
        image_root=image_root,
        evaluator_type="sem_seg",
        ignore_label=255,
        **metadata,
    )

In [3]:
#register_coco_instances("train-dataset", {}, RAW_DATA_PATH / "detection_coco.json", RAW_DATA_PATH / "images")
register_coco_panoptic_separated("train-dataset", {}, RAW_DATA_PATH / "images", RAW_DATA_PATH / "images", 
                                 "/tmp/placeholder/", RAW_DATA_PATH / "masks", RAW_DATA_PATH / "detection_coco.json",)

In [7]:
dataset_name = "train-dataset_separated"
metadata = MetadataCatalog.get(dataset_name)

In [10]:
MetadataCatalog.get(dataset_name).set(stuff_classes=["MainRailPolygon", "AlternativeRailPolygon", "Train"])

namespace(name='train-dataset_separated',
          panoptic_root=PosixPath('/workspace/ai-train/notebooks/../data/raw/images'),
          image_root=PosixPath('/workspace/ai-train/notebooks/../data/raw/images'),
          panoptic_json='/tmp/placeholder/',
          sem_seg_root=PosixPath('/workspace/ai-train/notebooks/../data/raw/masks'),
          json_file=PosixPath('/workspace/ai-train/notebooks/../data/raw/detection_coco.json'),
          evaluator_type='coco_panoptic_seg',
          ignore_label=255,
          stuff_classes=['MainRailPolygon', 'AlternativeRailPolygon', 'Train'],
          thing_classes=['Car',
                         'Human',
                         'Wagon',
                         'FacingSwitchL',
                         'FacingSwitchR',
                         'FacingSwitchNV',
                         'TrailingSwitchL',
                         'TrailingSwitchR',
                         'TrailingSwitchNV',
                         'SignalE',
           

In [9]:
dataset_dicts = DatasetCatalog.get(dataset_name)

In [13]:
metadata.stuff_classes

['MainRailPolygon', 'AlternativeRailPolygon', 'Train']

In [14]:
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1)
    out = visualizer.draw_dataset_dict(d)
    #out = visualizer.draw_sem_seg(d)
    plt.figure(figsize=(30, 20))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()

AssertionError: mask shape: (1512, 2688, 3), target dims: 1512, 2688